In [ ]:
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
import toml
import re
import urllib
from datetime import datetime
import string
import pygsheets
import requests
import time
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
import benchlingapi

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
from paulssonlab.api.util import base_url
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.golden_gate as golden_gate
import paulssonlab.cloning.codon as codon
import paulssonlab.cloning.optimization as optimization
import paulssonlab.api.twist as twist

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

In [ ]:
strain_sheet = gc.open_by_key(col["strains"]).worksheet()
plasmid_sheet = gc.open_by_key(col["plasmids"]).worksheet()
part_sheet = gc.open_by_key(col["parts"]).worksheet()
part_type_sheet = gc.open_by_key(col["parts"]).worksheet_by_title("Part types")

In [ ]:
drive_service = plasmid_sheet.client.drive.service
plasmid_folder = col["plasmid_maps"]
plasmid_maps = api.google.list_drive(drive_service, root=plasmid_folder)

# Twist API

Currently, this has to be run via the HMS network (or via the HMS proxy) because Twist restricts the IP address ranges that can access their API staging server.

In [ ]:
# HOW TO SPECIFY CDS START/END/WHETHER TO ADD STOP
# GG FLANKS
# HOW TO CHECK TWIST FLANKS?

# check where start codon is
# convert AA's -> nt's using E. coli codon usage table
# add stop codons
# add GG flanks (for integration into storage vector)
# check scores
# for all bad scores (AND all that contain RE sites), codon optimize
# codon optimize avoiding: BsaI, BsmBI, BbsI, AarI
# order options: 96-well plate, cap at 1000ug per well
# request quote
# shipping details: Johan/Paulsson/2135454219, share with email: paulssonlab@gmail.com, jqshenker+twist@g.harvard.edu
# checkout
# use existing payment method: BPO #70003066270
# confirm

# download plate map
# add plate map info to oLIB0.x spreadsheet (e.g., oLIB0.7.H12)

In [ ]:
TWIST_AUTHENTICATION_HEADERS = {
    "Authorization": "JWT {}".format(config["twist"]["api_token"]),
    "X-End-User-Token": config["twist"]["end_user_token"],
}

In [ ]:
r = requests.get(
    "https://twist-api.twistbioscience-staging.com/v1/prices/",
    headers=AUTHENTICATION_HEADERS,
)
twist_prices = r.json()

In [ ]:
twist_prices

## Codon optimization

In [ ]:
seqs = {
    "aaa5": (
        "cgtctcggtctcaaatgaccggtcatccggatgttgcagaaattgcagatctggcagaaggtctgctgccgaccacccgtaccaccgaagttcgtcagcatctggaaagctgtgaactgtgtgcagatgtttatgcaagcctgacagaaattcagggtctgctgggtaccctgcctgcaccggcaccgatgcctgatgatgttgcagcacgtattgatgcagcactggcagcagaaccgccgctgggtattgcagatggcacccgtgttagccgtgaaaccagcaccccggcagatagacctgcaggtcatgcacgtcctagcagcaccggtcctggtcgtaaagatcgtcgtcgtggtggtcgtcgtcgtattgcagttctgggtgcagttgcagcagcagccgcaattggtattggtagcgttgttgttagcagcctgaccgaagatagcagcagcggtaataccgcacgtgaacagcagaccgcaattgcagatacctttagcgaaggtcgtctgaaagatcgtgttaccaatctggttgcagatggtagcgcagaaaatggtagccgtaccccgcgtagctttggtatggaaagcgaaaatggtggtgaaaccgcagaaaatcatgttttcaaacagccgaccgttccggaatgtattcgtaaaggtattggtcgtgatgatgcagttattgcaaccgaaccgggtgtttataaaggtcgtgaagcactgctggttgttctgccggatgcaaccaatgatacccaggttaccgcatatattgttgaaaccgcatgtgttgatcagccggcagttggtaaagcaaagattctgctggaacatagctatgcacgtagctaataaggttgagaccggagacg",
        (14, 866),
    ),
    "bbb5": (
        "cgtctcggtctcaaatgagcgatggtaatgatgcacgtccggatgcaagagatggtcatgacccgggtagcgacgcacgtgatggccatgatcctggtagccatgcacgtgacggtcatgatccgggtgcaggtccgtttgatggtcacggtcctggtagcgatgcaggcgatggtgatgatgcaggtccggatgcacgtgatggtcatggtgcaccagcaggtcctgcaggtcgtgaacagcgtaaacatccgcatgatacacatgatatgcatgatacccacgatacccatgcaggtaaaggtaccgttaatcatggtcctgcagcacagggtcctgatgcaccgggtgctgatggtcctgcaaccgatgaagcagcactgcgtgcaatgatgcagcgtgcagttcgtgaaatggaaccgagcgatggtaccctggaacatctgcgtcgtgcagttccggctcgtcgtgcacgtaaacgtcaggcactggttggtgcagcagcagccgcactgtttctgggtaccgcagttccggcactggttcatgttagcaatgctaccggtgcaggcgcagatccgagcgttgcaggtaatgcaagccaggcacagggtggtgcaagccagggtaaagatccggctggtggtcagagcggtgttgcaggtaccggtgatacccctgaagatcgtgataaagcagatccgaaagaaaccccgggtggtaaagaaccgggtgctgcaaccggtgcaccacctagtggtgttccgagtgcaagcagtccggcagatgttcctgcatgtgcacctggtagcttaggtccggcagttgcaagcagcgcagaaccggatagcaccggtgttgtttatggtagctttcgtgttaccaatgttagcagcgatggttgtaccgttaccggtccgggtaccgttgttaccgcaagcttaggtgcagcagaagcaacccgtattggtaccgcacgtcatgcagcaggcgatgcagcagcaggattaccggatcctagtctggaaaccgcaagcctggcattagcaccgggtgcagcatatgaagttcagtttgcatgggttccgagcgaaacctgtccgaccacaggtggtacaacaggtggtggcagcggtggtccgagtcctgatcctagtcctaccgcagataccacagcagcaggcggtaccagtgcaggcggtggtgaagcaggtcctacaacccagctgattaccgaagatggtccggcagaaggtagcgttagcgttacctataccccggaaggtggtagcggtagcgcaaccgcaaccgttagcaatgcatgtgcaggtaccgtttattggaccggtctgctggcagatagcggtagcggtgcataataaggttgagaccggagacg",
        (14, 1382),
    ),
}

In [ ]:
# copied from swagger docs for POST /v1/users/{email}/codon-optimizations/
resp = requests.post(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/codon-optimizations/",
    json=[
        {
            "sequence": "ATGGCTTCTCCCATCTCAAATTCTTCTTCAACTCTCTACAGCTTATTTTTCTTTGGCCTCTTGCTTTCATTTTCGTTTGCTGGTCGTGCTCGAGCTAACCCCAATTACCGAGACGCCTTGGCTAAGTCGATATTGTTTTTTGAAGGACAACGCTCCGGTAGGATCCCGGCTAACCAACGGATCACTTGGAGGTCCAACTCCGGCCTCTACGATGGTGAACTTGCTCATGTGGATTTAACCGGCGGCTACTACGACGCCGGCGACAATGTAAAATTCAATCTTCCGATGGCTTTCACAACCACAATGCTTTCATGGGGAGCACTCGAGTACGGGGCGCGTATGGGTAGCGAATTAGGCAACACACGGGCCGCCATCCGTTGGGCCACCGATTACCTTCTCAAGTGCGCGACCGCCACTCCAGGCAAGCTCTACGTCGGCGTGGGAGACCCTCACGCCGACCACAAGTGCTGGGAACGGCCTGAGGACATGGATACTGTTCGAACCGTATACTCTGTTTCTGCCGGGAACCCAGGATCGGATGTTGCCGGAGAGACCGCGGCCGCACTGGCCGCCGCGTCGTTGGTGTTCCGACGAGTTGATAGGAAGTATTCACGGGTGTTGCTGGCGACGGCGAAGAAGGTGATGGAATTTGCGTTGGAGCACCGTGGATCGTATAGTGATTCGCTTTCCTCTGCTGTTTGTCCTTTTTATTGCTCTTATTCCGGATATAAGGATGAATTGGTATGGGGAGCAGCATGGCTTCTAAGAGCAACAAATGATGTTAAATACTTCAATTTGTTGAAGTCATTGGGAGGTGATGATGTGACTGATATCTTTAGTTGGGACAACAAATTTGCTGGTGCTCATGTCCTTTTGGGGAGATTGATGTTCAAGTTGCCAGAAAGTAACCTCCAATATGTGACATCCATAACGTTTTTGCTCACCACATATTCCAAATACATGTCTGCAGCCAAACACACATTCAACTGTGGCAACCTTGTCGTTACTCCAGCTTCTCTGAAAAACCTTGCTAAGATTCAGGTGGATTATATATTAGGAGTGAACCCATTGAAAATGTCATACATGGTGGGATATGGAAAGAACTTCCCAAAGAGAATTCATCATAGAGGATCTTCGCTGCCTTCCAAGGCCACCCACCCTCAAGCTATTGCCTGCGATGGTGGCTTCCAACCCTTCTTCTATTCCTACAACCCCAACCCTAATATCTTAATTGGCGCTGTCGTCGGCGGCCCCAATCAAAGTGATGGCTTCCCTGACGATCGCACTGATTACAGCCACTCTGAACCTGCTACGTACATCAATGCTGCTCTTGTTGGACCTCTTGCATTCTTCTCGGGCAAACATTGA",
            "external_id": "test_codon_opt3",
            "organism": "Homo sapiens",
            "avoid_introducing": ["EcoRI", "XbaI"],
            "optimization_start": 3,
            "optimization_len": 9,
        }
    ],
    headers=twist.auth_headers(config["twist"]),
)
resp.content

In [ ]:
# data = [
#     {
#         "sequence": "ATGGCTTCTCCCATCTCAAATTCTTCTTCAACTCTCTACAGCTTATTTTTCTTTGGCCTCTTGCTTTCATTTTCGTTTGCTGGTCGTGCTCGAGCTAACCCCAATTACCGAGACGCCTTGGCTAAGTCGATATTGTTTTTTGAAGGACAACGCTCCGGTAGGATCCCGGCTAACCAACGGATCACTTGGAGGTCCAACTCCGGCCTCTACGATGGTGAACTTGCTCATGTGGATTTAACCGGCGGCTACTACGACGCCGGCGACAATGTAAAATTCAATCTTCCGATGGCTTTCACAACCACAATGCTTTCATGGGGAGCACTCGAGTACGGGGCGCGTATGGGTAGCGAATTAGGCAACACACGGGCCGCCATCCGTTGGGCCACCGATTACCTTCTCAAGTGCGCGACCGCCACTCCAGGCAAGCTCTACGTCGGCGTGGGAGACCCTCACGCCGACCACAAGTGCTGGGAACGGCCTGAGGACATGGATACTGTTCGAACCGTATACTCTGTTTCTGCCGGGAACCCAGGATCGGATGTTGCCGGAGAGACCGCGGCCGCACTGGCCGCCGCGTCGTTGGTGTTCCGACGAGTTGATAGGAAGTATTCACGGGTGTTGCTGGCGACGGCGAAGAAGGTGATGGAATTTGCGTTGGAGCACCGTGGATCGTATAGTGATTCGCTTTCCTCTGCTGTTTGTCCTTTTTATTGCTCTTATTCCGGATATAAGGATGAATTGGTATGGGGAGCAGCATGGCTTCTAAGAGCAACAAATGATGTTAAATACTTCAATTTGTTGAAGTCATTGGGAGGTGATGATGTGACTGATATCTTTAGTTGGGACAACAAATTTGCTGGTGCTCATGTCCTTTTGGGGAGATTGATGTTCAAGTTGCCAGAAAGTAACCTCCAATATGTGACATCCATAACGTTTTTGCTCACCACATATTCCAAATACATGTCTGCAGCCAAACACACATTCAACTGTGGCAACCTTGTCGTTACTCCAGCTTCTCTGAAAAACCTTGCTAAGATTCAGGTGGATTATATATTAGGAGTGAACCCATTGAAAATGTCATACATGGTGGGATATGGAAAGAACTTCCCAAAGAGAATTCATCATAGAGGATCTTCGCTGCCTTCCAAGGCCACCCACCCTCAAGCTATTGCCTGCGATGGTGGCTTCCAACCCTTCTTCTATTCCTACAACCCCAACCCTAATATCTTAATTGGCGCTGTCGTCGGCGGCCCCAATCAAAGTGATGGCTTCCCTGACGATCGCACTGATTACAGCCACTCTGAACCTGCTACGTACATCAATGCTGCTCTTGTTGGACCTCTTGCATTCTTCTCGGGCAAACATTGA",
#         "external_id": "test_codon_opt",
#         "organism": "Homo sapiens",
#         "avoid_introducing": ["EcoRI", "XbaI"],
#         "optimization_start": 3,
#         "optimization_len": 9
#     }
# ]
data = [
    {
        "sequence": "ATGGCTTCTCCCATCTCAAATTCTTCTTCAACTCTCTACAGCTTATTTTTCTTTGGCCTCTTGCTTTCATTTTCGTTTGCTGGTCGTGCTCGAGCTAACCCCAATTACCGAGACGCCTTGGCTAAGTCGATATTGTTTTTTGAAGGACAACGCTCCGGTAGGATCCCGGCTAACCAACGGATCACTTGGAGGTCCAACTCCGGCCTCTACGATGGTGAACTTGCTCATGTGGATTTAACCGGCGGCTACTACGACGCCGGCGACAATGTAAAATTCAATCTTCCGATGGCTTTCACAACCACAATGCTTTCATGGGGAGCACTCGAGTACGGGGCGCGTATGGGTAGCGAATTAGGCAACACACGGGCCGCCATCCGTTGGGCCACCGATTACCTTCTCAAGTGCGCGACCGCCACTCCAGGCAAGCTCTACGTCGGCGTGGGAGACCCTCACGCCGACCACAAGTGCTGGGAACGGCCTGAGGACATGGATACTGTTCGAACCGTATACTCTGTTTCTGCCGGGAACCCAGGATCGGATGTTGCCGGAGAGACCGCGGCCGCACTGGCCGCCGCGTCGTTGGTGTTCCGACGAGTTGATAGGAAGTATTCACGGGTGTTGCTGGCGACGGCGAAGAAGGTGATGGAATTTGCGTTGGAGCACCGTGGATCGTATAGTGATTCGCTTTCCTCTGCTGTTTGTCCTTTTTATTGCTCTTATTCCGGATATAAGGATGAATTGGTATGGGGAGCAGCATGGCTTCTAAGAGCAACAAATGATGTTAAATACTTCAATTTGTTGAAGTCATTGGGAGGTGATGATGTGACTGATATCTTTAGTTGGGACAACAAATTTGCTGGTGCTCATGTCCTTTTGGGGAGATTGATGTTCAAGTTGCCAGAAAGTAACCTCCAATATGTGACATCCATAACGTTTTTGCTCACCACATATTCCAAATACATGTCTGCAGCCAAACACACATTCAACTGTGGCAACCTTGTCGTTACTCCAGCTTCTCTGAAAAACCTTGCTAAGATTCAGGTGGATTATATATTAGGAGTGAACCCATTGAAAATGTCATACATGGTGGGATATGGAAAGAACTTCCCAAAGAGAATTCATCATAGAGGATCTTCGCTGCCTTCCAAGGCCACCCACCCTCAAGCTATTGCCTGCGATGGTGGCTTCCAACCCTTCTTCTATTCCTACAACCCCAACCCTAATATCTTAATTGGCGCTGTCGTCGGCGGCCCCAATCAAAGTGATGGCTTCCCTGACGATCGCACTGATTACAGCCACTCTGAACCTGCTACGTACATCAATGCTGCTCTTGTTGGACCTCTTGCATTCTTCTCGGGCAAACATTGA",
        "external_id": "test_codon_opt",
        "organism": "Homo sapiens",
        "avoid_introducing": ["EcoRI", "XbaI"],
        "optimization_start": 3,
        "optimization_len": 9,
    }
]
b = twist._optimize_codons(config["twist"], data)

In [ ]:
b = twist.optimize_codons(config["twist"], seqs, avoid_enzymes=[])

In [ ]:
seqs = {
    "aaa5": "cgtctcggtctcaaatgaccggtcatccggatgttgcagaaattgcagatctggcagaaggtctgctgccgaccacccgtaccaccgaagttcgtcagcatctggaaagctgtgaactgtgtgcagatgtttatgcaagcctgacagaaattcagggtctgctgggtaccctgcctgcaccggcaccgatgcctgatgatgttgcagcacgtattgatgcagcactggcagcagaaccgccgctgggtattgcagatggcacccgtgttagccgtgaaaccagcaccccggcagatagacctgcaggtcatgcacgtcctagcagcaccggtcctggtcgtaaagatcgtcgtcgtggtggtcgtcgtcgtattgcagttctgggtgcagttgcagcagcagccgcaattggtattggtagcgttgttgttagcagcctgaccgaagatagcagcagcggtaataccgcacgtgaacagcagaccgcaattgcagatacctttagcgaaggtcgtctgaaagatcgtgttaccaatctggttgcagatggtagcgcagaaaatggtagccgtaccccgcgtagctttggtatggaaagcgaaaatggtggtgaaaccgcagaaaatcatgttttcaaacagccgaccgttccggaatgtattcgtaaaggtattggtcgtgatgatgcagttattgcaaccgaaccgggtgtttataaaggtcgtgaagcactgctggttgttctgccggatgcaaccaatgatacccaggttaccgcatatattgttgaaaccgcatgtgttgatcagccggcagttggtaaagcaaagattctgctggaacatagctatgcacgtagctaataaggttgagaccggagacg",
    "bbb5": "cgtctcggtctcaaatgagcgatggtaatgatgcacgtccggatgcaagagatggtcatgacccgggtagcgacgcacgtgatggccatgatcctggtagccatgcacgtgacggtcatgatccgggtgcaggtccgtttgatggtcacggtcctggtagcgatgcaggcgatggtgatgatgcaggtccggatgcacgtgatggtcatggtgcaccagcaggtcctgcaggtcgtgaacagcgtaaacatccgcatgatacacatgatatgcatgatacccacgatacccatgcaggtaaaggtaccgttaatcatggtcctgcagcacagggtcctgatgcaccgggtgctgatggtcctgcaaccgatgaagcagcactgcgtgcaatgatgcagcgtgcagttcgtgaaatggaaccgagcgatggtaccctggaacatctgcgtcgtgcagttccggctcgtcgtgcacgtaaacgtcaggcactggttggtgcagcagcagccgcactgtttctgggtaccgcagttccggcactggttcatgttagcaatgctaccggtgcaggcgcagatccgagcgttgcaggtaatgcaagccaggcacagggtggtgcaagccagggtaaagatccggctggtggtcagagcggtgttgcaggtaccggtgatacccctgaagatcgtgataaagcagatccgaaagaaaccccgggtggtaaagaaccgggtgctgcaaccggtgcaccacctagtggtgttccgagtgcaagcagtccggcagatgttcctgcatgtgcacctggtagcttaggtccggcagttgcaagcagcgcagaaccggatagcaccggtgttgtttatggtagctttcgtgttaccaatgttagcagcgatggttgtaccgttaccggtccgggtaccgttgttaccgcaagcttaggtgcagcagaagcaacccgtattggtaccgcacgtcatgcagcaggcgatgcagcagcaggattaccggatcctagtctggaaaccgcaagcctggcattagcaccgggtgcagcatatgaagttcagtttgcatgggttccgagcgaaacctgtccgaccacaggtggtacaacaggtggtggcagcggtggtccgagtcctgatcctagtcctaccgcagataccacagcagcaggcggtaccagtgcaggcggtggtgaagcaggtcctacaacccagctgattaccgaagatggtccggcagaaggtagcgttagcgttacctataccccggaaggtggtagcggtagcgcaaccgcaaccgttagcaatgcatgtgcaggtaccgtttattggaccggtctgctggcagatagcggtagcggtgcataataaggttgagaccggagacg",
}

In [ ]:
a = twist.score(config["twist"], seqs)

In [ ]:
a

In [ ]:
ids = [c["id"] for c in a]
url = f"{twist.API_URL}/v1/users/{config['twist']['ecommerce_username']}/constructs/describe/"
r = requests.get(
    url,
    json={"id__in": "b17096ca-f4dd-48b9-af94-14ffda8b1301", "scored": True},
    # data={"id__in": ",".join(ids)},
    headers=twist.auth_headers(config["twist"]),
)
(r.status_code, r.json())

In [ ]:
ids = [c["id"] for c in a]
url = f"{twist.API_URL}/v1/users/{config['twist']['ecommerce_username']}/constructs/describe/"
r = requests.get(
    url,
    json={"id__in": ",".join(ids), "scored": True},
    # data={"id__in": ",".join(ids)},
    headers=twist.auth_headers(config["twist"]),
)
(r.status_code, r.json())

In [ ]:
r = requests.get(
    # "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/constructs/describe/",
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/constructs/describe/",
    params={"id__in": ",".join(ids)},
    headers=twist.auth_headers(config["twist"]),
)

In [ ]:
r.json()

In [ ]:
url

## Scoring/codon optimization

In [ ]:
test_seq1 = sigma_sequences_to_order["ECF03_1198"]
test_seq2 = sigma_sequences_to_order["ECF11_987"]

In [ ]:
data = [
    {
        "sequences": [v],
        "name": k,
        "type": "NON_CLONED_GENE",
        "insertion_point_mes_uid": "na",
        "vector_mes_uid": "na",
    }
    for k, v in sigma_sequences_to_order.items()
]

In [ ]:
data = [
    {
        # "sequences": [test_seq1],
        "name": "test1",
        # "type": "NON_CLONED_GENE",
        # "insertion_point_mes_uid": "na",
        # "vector_mes_uid": "na",
    }
]

In [ ]:
data = [
    {
        "sequences": [
            "GAAGCAAAGAAAGATCAAAGGTGGGTAGCTGCAATGGAGGAGGAGTTGTCAATGATAGAAAAGAACAAAACTTGGATCCTTGTTGACAAACCTCAAGACATTAAGGTAATTGGAGTTAAATGGGTGTTTAGAACCAAGCTTAATGCTGATGGCTCGATTAATTAAGCTCAAAGCCAGGGTTATGGTTAAAGGGTATGCTCAAATCTTTAGTATTGATTGCACTGACACTTTTGCTCCTGTTGCTAGAATGGACAAAATCTGGTTATTGTTTGTAATAACTGCATAAAAGAGCTGGAAATTGTTTCAGTTGGATGTCAAATCAACTTTTTTAAATGGTGTCTTATAAGAAGAAATTTATGTTGAGCAGCCCGAAGGATGTGAGAAGCAAGGTGAAGGAAATAAAGTCTATCCTTCTCAAGAAGGCTCTTTATGGTCTAAAACAAGCTCCAAGAGCTTGGTATAGCAAAATTGATGAACATTTACTGAGCTTGGGATTTGTGAAAAGTCTATCAGAAACAAACTTGTATGTCAAGCACAATGGTACTGACATTCTTATTATTTCACTGTATGTTGATGACCTAGTAGTTATAGGAAACAATGCAGATCATATTCAAAATTTCAAATGGAAGATGATGAAGTTGTTTGAATGATAGATCTTGGGCTCATGTCCTACTTCCTTGGCATAGAGATCAAGCAAGGGCAAGGTGAAGTTTTCATCTACCAAAAAAATATGCAAAGGAAATACTAAAGAAGTTTAAGATGGACGAATGTAAGGCAGTAAGCACTCTAAAATGAATCAAAAAGAGAAGTTGTGCAAGGAAGATGGTGCTAACAAATTAAAGTTAATGACGGATATTTCAAGAGTTTAATTGGTTGCATGATATGTATCTCACAGCAACAAAGCCTAATATTTTGAATGTTGTGAGTATTTTGTCTCGTTTCATGCATTGTGCAAGTGAATTACATCTCAAGACAGCCAAAAGAGTGATTCGATATGTGAAAGGCACAAGTGATTTTGGTGTTAAGTTCACAAGGGGCAAGGAGTACTTCAAGCTAATTGGTTTTTCTAATAATGATTGGGGAGGTTCAGTTGATGATATGAGAAGCACTTCAGGCTACTGTTTTACTCTTGGCTCTGGTATTTTCTCTTGGAGCTCGAAGAAGCAAGAAATTGTAGCCCAATCCACTACTGAAGCAGAATTTATTGCTGCAATAGCTACTGCTAATCAAGCTTTATGGCCGAGGGAAATTTTACTTGATCTCAATCTAGAGCAGAAGGAAAGCATAGAGATTCTTGTTGATAACAAAGCTGCTATTGCTATTTCCCATAATCCTATGTTTTATAAGAAGACTAGATATTTTCACATCAAGTTATTCTTCCTAAGGTAAAAGCAGAAAAATGGAGAGGTGATCCTTGTCTGTCTACTGCAAAACAGAAGATCAAACCAGACATATTAACTAAATCGTT"
        ],
        "name": "LTR_3",
        "type": "NON_CLONED_GENE",
        "insertion_point_mes_uid": "na",
        "vector_mes_uid": "na",
    }
]

In [ ]:
data = [
    {
        "owner_contact": "string",
        "type": "string",
        "notes": "string",
        "name": "string",
        "sequences": {},
        "vector_mes_uid": "string",
        "vector_type": "string",
        "insertion_point_mes_uid": "string",
        "plate": 0,
        "row": 0,
        "column": 0,
        "adapters_on": True,
        "external_id": "string",
        "catalog_id": "string",
        "production_data": {},
        "product_code": "string",
    }
]

In [ ]:
r = requests.post(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/constructs/",
    data=dict(data=data),
    headers=TWIST_AUTHENTICATION_HEADERS,
)
(r.status_code, r.content)
# assert resp.status_code == 201

In [ ]:
ids = [construct["id"] for construct in json.loads(resp.content)][:500]
MAX_SLEEP_CYCLES = 60
for i in range(0, MAX_SLEEP_CYCLES):
    r = requests.get(
        "/v1/users/paulssonlab@gmail.com/constructs/describe/",
        data={"id__in": ",".join(ids), "scored": True},
    )
    scored_constructs = r.json()
    if len(scored_constructs) == len(ids):
        break
    time.sleep(5)

In [ ]:
import requests, time, json

tapi_headers = {
    "Authorization": "JWT " + settings.API_CLIENT_TOKEN,
    "X-End-User-Token": settings.END_USER_TOKEN,
}
resp = requests.post(
    "/v1/users/myuser@mycompanydomain.com/constructs/",
    data=[
        {
            "sequences": [
                "GAAGCAAAGAAAGATCAAAGGTGGGTAGCTGCAATGGAGGAGGAGTTGTCAATGATAGAAAAGAACAAAACTTGGATCCTTGTTGACAAACCTCAAGACATTAAGGTAATTGGAGTTAAATGGGTGTTTAGAACCAAGCTTAATGCTGATGGCTCGATTAATTAAGCTCAAAGCCAGGGTTATGGTTAAAGGGTATGCTCAAATCTTTAGTATTGATTGCACTGACACTTTTGCTCCTGTTGCTAGAATGGACAAAATCTGGTTATTGTTTGTAATAACTGCATAAAAGAGCTGGAAATTGTTTCAGTTGGATGTCAAATCAACTTTTTTAAATGGTGTCTTATAAGAAGAAATTTATGTTGAGCAGCCCGAAGGATGTGAGAAGCAAGGTGAAGGAAATAAAGTCTATCCTTCTCAAGAAGGCTCTTTATGGTCTAAAACAAGCTCCAAGAGCTTGGTATAGCAAAATTGATGAACATTTACTGAGCTTGGGATTTGTGAAAAGTCTATCAGAAACAAACTTGTATGTCAAGCACAATGGTACTGACATTCTTATTATTTCACTGTATGTTGATGACCTAGTAGTTATAGGAAACAATGCAGATCATATTCAAAATTTCAAATGGAAGATGATGAAGTTGTTTGAATGATAGATCTTGGGCTCATGTCCTACTTCCTTGGCATAGAGATCAAGCAAGGGCAAGGTGAAGTTTTCATCTACCAAAAAAATATGCAAAGGAAATACTAAAGAAGTTTAAGATGGACGAATGTAAGGCAGTAAGCACTCTAAAATGAATCAAAAAGAGAAGTTGTGCAAGGAAGATGGTGCTAACAAATTAAAGTTAATGACGGATATTTCAAGAGTTTAATTGGTTGCATGATATGTATCTCACAGCAACAAAGCCTAATATTTTGAATGTTGTGAGTATTTTGTCTCGTTTCATGCATTGTGCAAGTGAATTACATCTCAAGACAGCCAAAAGAGTGATTCGATATGTGAAAGGCACAAGTGATTTTGGTGTTAAGTTCACAAGGGGCAAGGAGTACTTCAAGCTAATTGGTTTTTCTAATAATGATTGGGGAGGTTCAGTTGATGATATGAGAAGCACTTCAGGCTACTGTTTTACTCTTGGCTCTGGTATTTTCTCTTGGAGCTCGAAGAAGCAAGAAATTGTAGCCCAATCCACTACTGAAGCAGAATTTATTGCTGCAATAGCTACTGCTAATCAAGCTTTATGGCCGAGGGAAATTTTACTTGATCTCAATCTAGAGCAGAAGGAAAGCATAGAGATTCTTGTTGATAACAAAGCTGCTATTGCTATTTCCCATAATCCTATGTTTTATAAGAAGACTAGATATTTTCACATCAAGTTATTCTTCCTAAGGTAAAAGCAGAAAAATGGAGAGGTGATCCTTGTCTGTCTACTGCAAAACAGAAGATCAAACCAGACATATTAACTAAATCGTT"
            ],
            "name": "LTR_3",
            "type": "NON_CLONED_GENE",
            "insertion_point_mes_uid": "na",
            "vector_mes_uid": "na",
        }
    ],
    headers=TWIST_AUTHENTICATION_HEADERS,
)
assert resp.status_code == 201
ids = [construct["id"] for construct in json.loads(resp.content)][:500]
MAX_SLEEP_CYCLES = 60
for i in range(0, MAX_SLEEP_CYCLES):
    resp = requests.get(
        "/v1/users/myuser@mycompanydomain.com/constructs/describe/",
        data={"id__in": ",".join(ids), "scored": True},
    )
    scored_constructs = json.loads(resp.content)
    if len(scored_constructs) == ids:
        break
    time.sleep(5)

## Past orders

In [ ]:
r = requests.get(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/orders/",
    headers=TWIST_AUTHENTICATION_HEADERS,
)
orders = r.json()

In [ ]:
[(o["order_name"], o["project_name"]) for o in orders]

## Twist debugging

In [ ]:
# copied from swagger docs for POST /v1/users/{email}/constructs/
resp = requests.post(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/constructs/",
    json=[
        {
            "sequences": [
                "GAAGCAAAGAAAGATCAAAGGTGGGTAGCTGCAATGGAGGAGGAGTTGTCAATGATAGAAAAGAACAAAACTTGGATCCTTGTTGACAAACCTCAAGACATTAAGGTAATTGGAGTTAAATGGGTGTTTAGAACCAAGCTTAATGCTGATGGCTCGATTAATTAAGCTCAAAGCCAGGGTTATGGTTAAAGGGTATGCTCAAATCTTTAGTATTGATTGCACTGACACTTTTGCTCCTGTTGCTAGAATGGACAAAATCTGGTTATTGTTTGTAATAACTGCATAAAAGAGCTGGAAATTGTTTCAGTTGGATGTCAAATCAACTTTTTTAAATGGTGTCTTATAAGAAGAAATTTATGTTGAGCAGCCCGAAGGATGTGAGAAGCAAGGTGAAGGAAATAAAGTCTATCCTTCTCAAGAAGGCTCTTTATGGTCTAAAACAAGCTCCAAGAGCTTGGTATAGCAAAATTGATGAACATTTACTGAGCTTGGGATTTGTGAAAAGTCTATCAGAAACAAACTTGTATGTCAAGCACAATGGTACTGACATTCTTATTATTTCACTGTATGTTGATGACCTAGTAGTTATAGGAAACAATGCAGATCATATTCAAAATTTCAAATGGAAGATGATGAAGTTGTTTGAATGATAGATCTTGGGCTCATGTCCTACTTCCTTGGCATAGAGATCAAGCAAGGGCAAGGTGAAGTTTTCATCTACCAAAAAAATATGCAAAGGAAATACTAAAGAAGTTTAAGATGGACGAATGTAAGGCAGTAAGCACTCTAAAATGAATCAAAAAGAGAAGTTGTGCAAGGAAGATGGTGCTAACAAATTAAAGTTAATGACGGATATTTCAAGAGTTTAATTGGTTGCATGATATGTATCTCACAGCAACAAAGCCTAATATTTTGAATGTTGTGAGTATTTTGTCTCGTTTCATGCATTGTGCAAGTGAATTACATCTCAAGACAGCCAAAAGAGTGATTCGATATGTGAAAGGCACAAGTGATTTTGGTGTTAAGTTCACAAGGGGCAAGGAGTACTTCAAGCTAATTGGTTTTTCTAATAATGATTGGGGAGGTTCAGTTGATGATATGAGAAGCACTTCAGGCTACTGTTTTACTCTTGGCTCTGGTATTTTCTCTTGGAGCTCGAAGAAGCAAGAAATTGTAGCCCAATCCACTACTGAAGCAGAATTTATTGCTGCAATAGCTACTGCTAATCAAGCTTTATGGCCGAGGGAAATTTTACTTGATCTCAATCTAGAGCAGAAGGAAAGCATAGAGATTCTTGTTGATAACAAAGCTGCTATTGCTATTTCCCATAATCCTATGTTTTATAAGAAGACTAGATATTTTCACATCAAGTTATTCTTCCTAAGGTAAAAGCAGAAAAATGGAGAGGTGATCCTTGTCTGTCTACTGCAAAACAGAAGATCAAACCAGACATATTAACTAAATCGTT"
            ],
            "name": "LTR_3",
            "type": "NON_CLONED_GENE",
            "insertion_point_mes_uid": "na",
            "vector_mes_uid": "na",
        }
    ],
    headers=twist.auth_headers(config["twist"]),
)
resp.content

In [ ]:
# copied from swagger docs for POST /v1/users/{email}/codon-optimizations/
resp = requests.post(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/codon-optimizations/",
    json=[
        {
            "sequence": "ATGGCTTCTCCCATCTCAAATTCTTCTTCAACTCTCTACAGCTTATTTTTCTTTGGCCTCTTGCTTTCATTTTCGTTTGCTGGTCGTGCTCGAGCTAACCCCAATTACCGAGACGCCTTGGCTAAGTCGATATTGTTTTTTGAAGGACAACGCTCCGGTAGGATCCCGGCTAACCAACGGATCACTTGGAGGTCCAACTCCGGCCTCTACGATGGTGAACTTGCTCATGTGGATTTAACCGGCGGCTACTACGACGCCGGCGACAATGTAAAATTCAATCTTCCGATGGCTTTCACAACCACAATGCTTTCATGGGGAGCACTCGAGTACGGGGCGCGTATGGGTAGCGAATTAGGCAACACACGGGCCGCCATCCGTTGGGCCACCGATTACCTTCTCAAGTGCGCGACCGCCACTCCAGGCAAGCTCTACGTCGGCGTGGGAGACCCTCACGCCGACCACAAGTGCTGGGAACGGCCTGAGGACATGGATACTGTTCGAACCGTATACTCTGTTTCTGCCGGGAACCCAGGATCGGATGTTGCCGGAGAGACCGCGGCCGCACTGGCCGCCGCGTCGTTGGTGTTCCGACGAGTTGATAGGAAGTATTCACGGGTGTTGCTGGCGACGGCGAAGAAGGTGATGGAATTTGCGTTGGAGCACCGTGGATCGTATAGTGATTCGCTTTCCTCTGCTGTTTGTCCTTTTTATTGCTCTTATTCCGGATATAAGGATGAATTGGTATGGGGAGCAGCATGGCTTCTAAGAGCAACAAATGATGTTAAATACTTCAATTTGTTGAAGTCATTGGGAGGTGATGATGTGACTGATATCTTTAGTTGGGACAACAAATTTGCTGGTGCTCATGTCCTTTTGGGGAGATTGATGTTCAAGTTGCCAGAAAGTAACCTCCAATATGTGACATCCATAACGTTTTTGCTCACCACATATTCCAAATACATGTCTGCAGCCAAACACACATTCAACTGTGGCAACCTTGTCGTTACTCCAGCTTCTCTGAAAAACCTTGCTAAGATTCAGGTGGATTATATATTAGGAGTGAACCCATTGAAAATGTCATACATGGTGGGATATGGAAAGAACTTCCCAAAGAGAATTCATCATAGAGGATCTTCGCTGCCTTCCAAGGCCACCCACCCTCAAGCTATTGCCTGCGATGGTGGCTTCCAACCCTTCTTCTATTCCTACAACCCCAACCCTAATATCTTAATTGGCGCTGTCGTCGGCGGCCCCAATCAAAGTGATGGCTTCCCTGACGATCGCACTGATTACAGCCACTCTGAACCTGCTACGTACATCAATGCTGCTCTTGTTGGACCTCTTGCATTCTTCTCGGGCAAACATTGA",
            "external_id": "test_codon_opt",
            "organism": "Homo sapiens",
            "avoid_introducing": ["EcoRI", "XbaI"],
            "optimization_start": 3,
            "optimization_len": 9,
        }
    ],
    headers=TWIST_AUTHENTICATION_HEADERS,
)
resp.content

In [ ]:
# copied from swagger docs for POST /v1/users/{email}/codon-optimizations/
resp = requests.post(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/codon-optimizations/",
    json=[
        {
            "sequence": "ATGGCTTCTCCCATCTCAAATTCTTCTTCAACTCTCTACAGCTTATTTTTCTTTGGCCTCTTGCTTTCATTTTCGTTTGCTGGTCGTGCTCGAGCTAACCCCAATTACCGAGACGCCTTGGCTAAGTCGATATTGTTTTTTGAAGGACAACGCTCCGGTAGGATCCCGGCTAACCAACGGATCACTTGGAGGTCCAACTCCGGCCTCTACGATGGTGAACTTGCTCATGTGGATTTAACCGGCGGCTACTACGACGCCGGCGACAATGTAAAATTCAATCTTCCGATGGCTTTCACAACCACAATGCTTTCATGGGGAGCACTCGAGTACGGGGCGCGTATGGGTAGCGAATTAGGCAACACACGGGCCGCCATCCGTTGGGCCACCGATTACCTTCTCAAGTGCGCGACCGCCACTCCAGGCAAGCTCTACGTCGGCGTGGGAGACCCTCACGCCGACCACAAGTGCTGGGAACGGCCTGAGGACATGGATACTGTTCGAACCGTATACTCTGTTTCTGCCGGGAACCCAGGATCGGATGTTGCCGGAGAGACCGCGGCCGCACTGGCCGCCGCGTCGTTGGTGTTCCGACGAGTTGATAGGAAGTATTCACGGGTGTTGCTGGCGACGGCGAAGAAGGTGATGGAATTTGCGTTGGAGCACCGTGGATCGTATAGTGATTCGCTTTCCTCTGCTGTTTGTCCTTTTTATTGCTCTTATTCCGGATATAAGGATGAATTGGTATGGGGAGCAGCATGGCTTCTAAGAGCAACAAATGATGTTAAATACTTCAATTTGTTGAAGTCATTGGGAGGTGATGATGTGACTGATATCTTTAGTTGGGACAACAAATTTGCTGGTGCTCATGTCCTTTTGGGGAGATTGATGTTCAAGTTGCCAGAAAGTAACCTCCAATATGTGACATCCATAACGTTTTTGCTCACCACATATTCCAAATACATGTCTGCAGCCAAACACACATTCAACTGTGGCAACCTTGTCGTTACTCCAGCTTCTCTGAAAAACCTTGCTAAGATTCAGGTGGATTATATATTAGGAGTGAACCCATTGAAAATGTCATACATGGTGGGATATGGAAAGAACTTCCCAAAGAGAATTCATCATAGAGGATCTTCGCTGCCTTCCAAGGCCACCCACCCTCAAGCTATTGCCTGCGATGGTGGCTTCCAACCCTTCTTCTATTCCTACAACCCCAACCCTAATATCTTAATTGGCGCTGTCGTCGGCGGCCCCAATCAAAGTGATGGCTTCCCTGACGATCGCACTGATTACAGCCACTCTGAACCTGCTACGTACATCAATGCTGCTCTTGTTGGACCTCTTGCATTCTTCTCGGGCAAACATTGA",
            "external_id": "test_codon_opt",
            "organism": "Homo sapiens",
            "avoid_introducing": ["EcoRI", "XbaI"],
            "optimization_start": 3,
            "optimization_len": 9,
        }
    ],
    headers=TWIST_AUTHENTICATION_HEADERS,
)
resp.content

In [ ]:
# copied from swagger docs for POST /v1/users/{email}/constructs/
resp = requests.post(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/constructs/",
    data=dict(
        data=[
            {
                "sequences": [
                    "GAAGCAAAGAAAGATCAAAGGTGGGTAGCTGCAATGGAGGAGGAGTTGTCAATGATAGAAAAGAACAAAACTTGGATCCTTGTTGACAAACCTCAAGACATTAAGGTAATTGGAGTTAAATGGGTGTTTAGAACCAAGCTTAATGCTGATGGCTCGATTAATTAAGCTCAAAGCCAGGGTTATGGTTAAAGGGTATGCTCAAATCTTTAGTATTGATTGCACTGACACTTTTGCTCCTGTTGCTAGAATGGACAAAATCTGGTTATTGTTTGTAATAACTGCATAAAAGAGCTGGAAATTGTTTCAGTTGGATGTCAAATCAACTTTTTTAAATGGTGTCTTATAAGAAGAAATTTATGTTGAGCAGCCCGAAGGATGTGAGAAGCAAGGTGAAGGAAATAAAGTCTATCCTTCTCAAGAAGGCTCTTTATGGTCTAAAACAAGCTCCAAGAGCTTGGTATAGCAAAATTGATGAACATTTACTGAGCTTGGGATTTGTGAAAAGTCTATCAGAAACAAACTTGTATGTCAAGCACAATGGTACTGACATTCTTATTATTTCACTGTATGTTGATGACCTAGTAGTTATAGGAAACAATGCAGATCATATTCAAAATTTCAAATGGAAGATGATGAAGTTGTTTGAATGATAGATCTTGGGCTCATGTCCTACTTCCTTGGCATAGAGATCAAGCAAGGGCAAGGTGAAGTTTTCATCTACCAAAAAAATATGCAAAGGAAATACTAAAGAAGTTTAAGATGGACGAATGTAAGGCAGTAAGCACTCTAAAATGAATCAAAAAGAGAAGTTGTGCAAGGAAGATGGTGCTAACAAATTAAAGTTAATGACGGATATTTCAAGAGTTTAATTGGTTGCATGATATGTATCTCACAGCAACAAAGCCTAATATTTTGAATGTTGTGAGTATTTTGTCTCGTTTCATGCATTGTGCAAGTGAATTACATCTCAAGACAGCCAAAAGAGTGATTCGATATGTGAAAGGCACAAGTGATTTTGGTGTTAAGTTCACAAGGGGCAAGGAGTACTTCAAGCTAATTGGTTTTTCTAATAATGATTGGGGAGGTTCAGTTGATGATATGAGAAGCACTTCAGGCTACTGTTTTACTCTTGGCTCTGGTATTTTCTCTTGGAGCTCGAAGAAGCAAGAAATTGTAGCCCAATCCACTACTGAAGCAGAATTTATTGCTGCAATAGCTACTGCTAATCAAGCTTTATGGCCGAGGGAAATTTTACTTGATCTCAATCTAGAGCAGAAGGAAAGCATAGAGATTCTTGTTGATAACAAAGCTGCTATTGCTATTTCCCATAATCCTATGTTTTATAAGAAGACTAGATATTTTCACATCAAGTTATTCTTCCTAAGGTAAAAGCAGAAAAATGGAGAGGTGATCCTTGTCTGTCTACTGCAAAACAGAAGATCAAACCAGACATATTAACTAAATCGTT"
                ],
                "name": "LTR_3",
                "type": "NON_CLONED_GENE",
                "insertion_point_mes_uid": "na",
                "vector_mes_uid": "na",
            }
        ]
    ),
    headers=TWIST_AUTHENTICATION_HEADERS,
)
resp.content

In [ ]:
# copied from swagger docs for POST /v1/users/{email}/codon-optimizations/
resp = requests.post(
    "https://twist-api.twistbioscience-staging.com/v1/users/paulssonlab@gmail.com/codon-optimizations/",
    data=dict(
        data=[
            {
                "sequence": "ATGGCTTCTCCCATCTCAAATTCTTCTTCAACTCTCTACAGCTTATTTTTCTTTGGCCTCTTGCTTTCATTTTCGTTTGCTGGTCGTGCTCGAGCTAACCCCAATTACCGAGACGCCTTGGCTAAGTCGATATTGTTTTTTGAAGGACAACGCTCCGGTAGGATCCCGGCTAACCAACGGATCACTTGGAGGTCCAACTCCGGCCTCTACGATGGTGAACTTGCTCATGTGGATTTAACCGGCGGCTACTACGACGCCGGCGACAATGTAAAATTCAATCTTCCGATGGCTTTCACAACCACAATGCTTTCATGGGGAGCACTCGAGTACGGGGCGCGTATGGGTAGCGAATTAGGCAACACACGGGCCGCCATCCGTTGGGCCACCGATTACCTTCTCAAGTGCGCGACCGCCACTCCAGGCAAGCTCTACGTCGGCGTGGGAGACCCTCACGCCGACCACAAGTGCTGGGAACGGCCTGAGGACATGGATACTGTTCGAACCGTATACTCTGTTTCTGCCGGGAACCCAGGATCGGATGTTGCCGGAGAGACCGCGGCCGCACTGGCCGCCGCGTCGTTGGTGTTCCGACGAGTTGATAGGAAGTATTCACGGGTGTTGCTGGCGACGGCGAAGAAGGTGATGGAATTTGCGTTGGAGCACCGTGGATCGTATAGTGATTCGCTTTCCTCTGCTGTTTGTCCTTTTTATTGCTCTTATTCCGGATATAAGGATGAATTGGTATGGGGAGCAGCATGGCTTCTAAGAGCAACAAATGATGTTAAATACTTCAATTTGTTGAAGTCATTGGGAGGTGATGATGTGACTGATATCTTTAGTTGGGACAACAAATTTGCTGGTGCTCATGTCCTTTTGGGGAGATTGATGTTCAAGTTGCCAGAAAGTAACCTCCAATATGTGACATCCATAACGTTTTTGCTCACCACATATTCCAAATACATGTCTGCAGCCAAACACACATTCAACTGTGGCAACCTTGTCGTTACTCCAGCTTCTCTGAAAAACCTTGCTAAGATTCAGGTGGATTATATATTAGGAGTGAACCCATTGAAAATGTCATACATGGTGGGATATGGAAAGAACTTCCCAAAGAGAATTCATCATAGAGGATCTTCGCTGCCTTCCAAGGCCACCCACCCTCAAGCTATTGCCTGCGATGGTGGCTTCCAACCCTTCTTCTATTCCTACAACCCCAACCCTAATATCTTAATTGGCGCTGTCGTCGGCGGCCCCAATCAAAGTGATGGCTTCCCTGACGATCGCACTGATTACAGCCACTCTGAACCTGCTACGTACATCAATGCTGCTCTTGTTGGACCTCTTGCATTCTTCTCGGGCAAACATTGA",
                "external_id": "test_codon_opt",
                "organism": "Homo sapiens",
                "avoid_introducing": ["EcoRI", "XbaI"],
                "optimization_start": 3,
                "optimization_len": 9,
            }
        ]
    ),
    headers=TWIST_AUTHENTICATION_HEADERS,
)
resp.content